In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import requests
import json
import nest_asyncio
import sys
import io
import os
import builtins
from fpdf import FPDF

In [12]:
import IPython
class SafeOutput(io.TextIOBase):
    def __init__(self, wrapped):
        self._wrapped = wrapped

    def write(self, s):
        try:
            return self._wrapped.write(s)
        except UnicodeEncodeError:
            s = s.encode('utf-8', 'replace').decode('utf-8')
            return self._wrapped.write(s)

    def flush(self):
        return self._wrapped.flush()

sys.stdout = SafeOutput(sys.__stdout__)
sys.stderr = SafeOutput(sys.__stderr__)

nest_asyncio.apply()

In [3]:
data = {
    "text": [
        "Skilled in Python, pandas, ML, data visualization",
        "Built responsive websites using HTML, CSS, JavaScript",
        "Worked with React and Node.js for full stack development",
        "Used TensorFlow and PyTorch for NLP and computer vision tasks",
        "Experience in Kubernetes, Docker, AWS and CI/CD pipelines"
    ],
    "label": [
        "Data Scientist",
        "Frontend Developer",
        "Full Stack Developer",
        "AI Engineer",
        "DevOps Engineer"
    ]
}

df = pd.DataFrame(data)

In [13]:
le = LabelEncoder()
df["label_encoded"] = le.fit_transform(df["label"])

vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df["text"])
y = df["label_encoded"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [15]:
def predict_job_role(resume_text):
    vec = vectorizer.transform([resume_text])
    pred = model.predict(vec)
    return le.inverse_transform(pred)[0]

In [16]:
api_key = "your_rapidapi_key_here"  # ⚠️ Replace with your actual key

def fetch_jobs(query, location="India", num_results=5):
    url = "https://jsearch.p.rapidapi.com/search"

    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "jsearch.p.rapidapi.com"
    }

    params = {
        "query": query,
        "location": location,
        "page": "1",
        "num_pages": "1"
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        jobs = response.json().get("data", [])[:num_results]
    except Exception as e:
        print("❌ Error fetching jobs:", e)
        return []

    return [
        {
            "title": job.get("job_title", "N/A"),
            "company": job.get("employer_name", "N/A"),
            "location": job.get("job_city", "N/A"),
            "description": job.get("job_description", "")[:200],
            "apply_link": job.get("job_apply_link", "N/A")
        }
        for job in jobs
    ]


In [17]:
def generate_resume(name, skills, experience, role):
    return f"""
Name: {name}
Role: {role}

Summary:
Passionate {role.lower()} skilled in {', '.join(skills)}. Adept at solving problems using innovative solutions with a proven track record of delivering results.

Skills:
{', '.join(skills)}

Experience:
{experience}

Education:
Bachelor’s in Computer Science or relevant field
"""

def generate_cover_letter(name, role, company):
    return f"""
Dear Hiring Manager,

I am writing to express my interest in the {role} position at {company}. With a strong background in {role.lower()} skills and hands-on experience in real-world projects, I believe I can contribute significantly to your team.

I am passionate about continuous learning and leveraging technology to solve complex challenges. I would welcome the opportunity to bring my enthusiasm and dedication to {company}.

Thank you for considering my application.

Sincerely,
{name}
"""


def export_to_pdf(text, filename="document.pdf"):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    for line in text.split('\n'):
        pdf.multi_cell(0, 10, line)
    pdf.output(filename)
    print(f"📄 Exported to {filename}")


In [10]:
def get_matched_jobs_for_resume(resume_text, user_name="Your Name", experience="2+ years of industry experience", generate_docs=True):
    predicted_role = predict_job_role(resume_text)
    print(f"🔍 Predicted Job Role: {predicted_role}\n")

    matched_jobs = fetch_jobs(predicted_role)

    if not matched_jobs:
        print("❌ No jobs found.")
    else:
        for idx, job in enumerate(matched_jobs, 1):
            print(f"🔹 Job #{idx}: {job['title']} at {job['company']} ({job['location']})")
            print(f"📝 {job['description']}...")
            print(f"✅ Apply here: {job['apply_link']}\n")

            if generate_docs:
                skills = resume_text.split(", ")
                resume = generate_resume(user_name, skills, experience, predicted_role)
                cover_letter = generate_cover_letter(user_name, predicted_role, job['company'])
                print("📄 Generated Resume:\n", resume)
                print("✉️ Generated Cover Letter:\n", cover_letter)
                print("="*80)

In [18]:
def get_matched_jobs_for_resume(resume_text, user_name="Your Name", experience="2+ years of industry experience", generate_docs=True):
    predicted_role = predict_job_role(resume_text)
    print(f"🔍 Predicted Job Role: {predicted_role}\n")

    matched_jobs = fetch_jobs(predicted_role)

    if not matched_jobs:
        print("❌ No jobs found.")
    else:
        for idx, job in enumerate(matched_jobs, 1):
            print(f"🔹 Job #{idx}: {job['title']} at {job['company']} ({job['location']})")
            print(f"📝 {job['description']}...")
            print(f"✅ Apply here: {job['apply_link']}\n")

            if generate_docs:
                skills = [s.strip() for s in resume_text.replace("I have experience with", "").replace("and", ",").split(",") if s.strip()]
                resume = generate_resume(user_name, skills, experience, predicted_role)
                cover_letter = generate_cover_letter(user_name, predicted_role, job['company'])

                print("📄 Generated Resume:\n", resume)
                print("✉️ Generated Cover Letter:\n", cover_letter)
                print("="*80)

                export_to_pdf(resume, filename=f"resume_{user_name.replace(' ', '_')}.pdf")
                export_to_pdf(cover_letter, filename=f"cover_letter_{user_name.replace(' ', '_')}_{job['company'].replace(' ', '_')}.pdf")


In [22]:
resume_sample = "I have experience with Python, pandas, data analysis, machine learning models, and data visualization tools like Matplotlib and Seaborn."
get_matched_jobs_for_resume(resume_sample, user_name="Rohit Sharma")


NameError: name 'FPDF' is not defined